In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

url = 'https://canpl.ca/statistics/teams/2022/'

excel_filename = 'CPL_2022.xlsx'

chrome_options = Options()  
chrome_options.add_argument("--headless") # Opens the browser up in background

browser = Chrome(options=chrome_options, executable_path='C:\ChromeDriver\chromedriver118\chromedriver.exe')
browser.get(url)
timeout = 5
try:
     element_present = EC.presence_of_element_located((By.TAG_NAME, 'table'))
     WebDriverWait(browser, timeout).until(element_present)
     html = browser.page_source
     page_soup = BeautifulSoup(html, 'html.parser')
     table = page_soup.findAll('table', {'class': 'stats-data-table table'})
except TimeoutException:
     print("Timed out waiting for page to load")

In [2]:
theader = table[0].find('thead')
headers = []
for i in theader.find_all('th'):
    title = i.text
    headers.append(title)
# removed the first two columns because they are not needed
headers = headers[2:]

In [3]:
def get_team_name(team):
    for team_name in team:
        if 'cav' in team_name:
            return 'Calvary FC'
        elif 'ato' in team_name:
            return 'Atletico Ottawa'
        elif 'for' in team_name:
            return 'Forge FC'
        elif 'hfx' in team_name:
            return 'HFX Wanderers FC'
        elif 'pac' in team_name:
            return 'Pacific FC'
        elif 'val' in team_name:
            return 'Valour FC'
        elif 'yrk' in team_name:
            return 'York'
        elif 'vfc' in team_name:
            return 'Vancouver FC'

tbody = table[0].find('tbody')
players = []
for tr in tbody.find_all('tr'):
    row_data = tr.find_all('td')
    player = [get_team_name(i.find_all('div')[0]['class']) if len(i.find_all('div')) > 0 else i.text for i in row_data]
    players.append(player[2:])

In [4]:
import pandas as pd
df = pd.DataFrame(players, columns=headers)
df.to_excel(excel_filename)